In [1]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
#read the new_csv
car_df= pd.read_csv('./preprocessedCombineCategories.csv')

In [3]:
featureName= list(car_df.columns.values) #get the column/feature name of the data
#handle the ambiguity in the data
dayIndex= car_df.shape[0] + 2   #set the value other than 0-15419
monthIndex= car_df.shape[0] + 1 #set the value other than 0-15419

for i in range(car_df.shape[0]):
    if(car_df['DayOfWeekClaimed'][i]=='0'):
        dayIndex= i
        print("index: ", i)
    if(car_df['MonthClaimed'][i]=='0'):
        monthIndex= i
        print("\nindex:",i)

#create dataframe for keeping new form of dataset
carData= pd.DataFrame(columns=featureName)
j= 0
#put all the data into new dataframe
for i in range(car_df.shape[0]):
    if(i!=dayIndex):
        carData.loc[j]= car_df.loc[i]
        j += 1

#replace map
replace_Month = {'Month':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

replace_MonthClaimed = {'MonthClaimed':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

carData.replace(replace_Month,inplace=True)
carData.replace(replace_MonthClaimed,inplace=True)

#function to calculate the no of days passed between the accident and the claims.
# Reporting Gap:
def get_day(year,month,weekOfMonth,dayOfWeek):
    count = 0
    c = calendar.TextCalendar(calendar.SUNDAY)
    l = []
    year= int(year)
    month=int(month)
    for i in c.itermonthdates(year,month):
        l.append(i)
    for j in range(len(l)):
        day = calendar.day_name[l[j].weekday()]
        if day == dayOfWeek:
            count += 1
        if count == weekOfMonth:
            return l[j]
            break

def differ_days(date1,date2):
    a = date1
    b = date2
    return (a - b).days



index:  1516

index: 1516


In [4]:
day_diff = np.zeros((carData.shape[0],1))
i = 0
for i in range(carData.shape[0]):

    if(carData['MonthClaimed'][i]-carData['Month'][i]) < 0:
        year2 = carData['Year'][i] + 1
        month2 = carData['MonthClaimed'][i]
        week2 = carData['WeekOfMonthClaimed'][i]
        day2 = carData['DayOfWeekClaimed'][i]
        year1 = carData['Year'][i]
        month1 = carData['Month'][i]
        week1 = carData['WeekOfMonth'][i]
        day1 = carData['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
    else:
        year2 = carData['Year'][i]
        month2 = carData['MonthClaimed'][i]
        week2 = carData['WeekOfMonthClaimed'][i]
        day2 = carData['DayOfWeekClaimed'][i]
        year1 = carData['Year'][i]
        month1 = carData['Month'][i]
        week1 = carData['WeekOfMonth'][i]
        day1 = carData['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))

#adding column to the existing dataframe
carData['daysDiff'] = day_diff 
carData.head(3)


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,daysDiff
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,26 to 30,No,No,External,more than 30,0 to 3 years,more than 1,1994,Liability,6.0
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,31 to 35,Yes,No,External,more than 30,no change,1 vehicle,1994,Collision,5.0
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,41 to 50,No,No,External,more than 30,no change,1 vehicle,1994,Collision,13.0


In [5]:
#now drop the original attibutes, like 'Month' column(we don't need anymore)
carData.drop(['Month'],axis=1,inplace=True)
carData.drop(['MonthClaimed'],axis=1,inplace=True)
carData.drop(['DayOfWeek'],axis=1,inplace=True)
carData.drop(['DayOfWeekClaimed'],axis=1,inplace=True)
carData.drop(['WeekOfMonth'],inplace=True,axis=1)
carData.drop(['WeekOfMonthClaimed'],inplace=True,axis=1)
carData.drop(['PolicyType'],axis=1,inplace=True)
carData.drop(['PolicyNumber'],inplace=True,axis=1)
carData.drop(['RepNumber'],axis=1,inplace=True)
carData.drop(['Deductible'],inplace=True,axis=1)
carData.drop(['AgeOfPolicyHolder'],inplace=True,axis=1)
carData.drop(['Year'],inplace=True,axis=1)


In [6]:
carData.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehicleCategory,VehiclePrice,FraudFound,DriverRating,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,Sport,"more than 69,000",No,1,...,more than 30,less than 4 years,No,No,External,more than 30,0 to 3 years,more than 1,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,Sport,"more than 69,000",No,4,...,more than 30,4 to 6 years,Yes,No,External,more than 30,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,Sport,"more than 69,000",No,3,...,1,7 years,No,No,External,more than 30,no change,1 vehicle,Collision,13.0
3,Honda,Rural,Male,Married,65,Third Party,Sport,"20,000 to 39,000",No,2,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,Sport,"more than 69,000",No,1,...,more than 30,4 to 6 years,No,No,External,more than 30,no change,1 vehicle,Collision,15.0


In [7]:
#get the lable of the datasets
label_Number = LabelEncoder()  #object of lable encoder

#conver label to Number
carData['FraudFound'] = label_Number.fit_transform(carData['FraudFound'].astype('str'))

yLable= carData['FraudFound']

#drop the lable from the dataset
carData.drop(['FraudFound'],inplace=True,axis=1)


In [8]:
#change the data type to numeric
carData['Age']= pd.to_numeric(carData['Age'])
carData['DriverRating']= pd.to_numeric(carData['DriverRating'])
carData['daysDiff']= pd.to_numeric(carData['daysDiff'])


In [9]:
#select all the attributes of type object
carObject= carData.select_dtypes(include=['object']).copy()

#drop the attributes of type object
carData.drop(carData.select_dtypes(['object']),inplace=True,axis=1)

#normalization of feature to bring the value in the range [0,1]
minMaxScale= MinMaxScaler() #minMax scaler
carData= minMaxScale.fit_transform(carData)

#converting numpyarry to dataframe
carData= pd.DataFrame(carData)


/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [10]:
#one hot encoding
car_copy = carObject.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['VehicleCategory'],prefix=['VehicleCategory'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])
car_copy = pd.get_dummies(car_copy,columns=['BasePolicy'],prefix=['BasePolicy'])

In [11]:
#putt all the attributes together
carObject= car_copy
carObject['Age']= carData[0]
carObject['DriverRating']= carData[1]
carObject['DaysDiff']= carData[2]
carObject['Lable']= yLable

In [12]:
#get the lable from the dataset
carLable= carObject['Lable']
carObject.drop(['Lable'],inplace=True,axis=1) #drop the lable;


In [13]:
carDateNormalized= carObject

In [14]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1

In [15]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 56)
(923, 56)


,Make_Ford,Make_Honda,Make_Mercury,Make_VW,AccidentArea_Rural,AccidentArea_Urban,Sex_Female,Sex_Male,MartalStatus_Divorced,MartalStatus_Married,...,AddressChange-Claim_4 to 8 years,AddressChange-Claim_no change,NumberOfCars_1 vehicle,NumberOfCars_more than 1,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,DriverRating,DaysDiff
0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.078125,0.000000,0.071795
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.281250,1.000000,0.069231
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.484375,0.666667,0.089744
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.765625,0.333333,0.107692
4,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.171875,0.000000,0.094872


In [16]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [17]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the trainset into trainset and validation set
nonFraudX_trainNew,nonFraudX_Valid,nonFraudY_trainNew,nonFraudY_Valid= train_test_split(nonFraudX_train,nonFraudY_train,random_state=3,test_size=0.2)



nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 56)


In [18]:
#convert into numpy
nonFraudX_trainNew= nonFraudX_trainNew.values
nonFraudY_trainNew= nonFraudY_trainNew.values

nonFraudX_test= nonFraudX_test.values
nonFraudY_test= nonFraudY_test.values

fraudFeatures= fraudulent.values
fraudLabel= fraudulentLable.values

nonFraudX_Valid= nonFraudX_Valid.values
nonFraudY_Valid= nonFraudY_Valid.values

In [19]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [20]:
#make the  nonfraudulent dataset for train and test loader
myNonFraudulentTrainDataset= myDataset(nonFraudX_trainNew,nonFraudY_trainNew)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make validation loader
myNonFraudulentValidation= myDataset(nonFraudX_Valid,nonFraudY_Valid)

#make the fraudulent dataset for train and test loader
myFraudulentDataset= myDataset(fraudFeatures,fraudLabel)


In [21]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=4,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentValidationLoader= torch.utils.data.DataLoader(myNonFraudulentValidation,batch_size=4,shuffle=True,num_workers=0)
#make the trainloader and test loader for nonfraudulent dataset.
fraudulentLoader= torch.utils.data.DataLoader(myFraudulentDataset,batch_size=1,shuffle=True,num_workers=0)




In [22]:
#Network architecture for the base autoencoders
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(56,42),
            nn.ReLU(),
            nn.Linear(42,28),
            nn.LeakyReLU(),
            nn.Linear(28,14),
            nn.Sigmoid(),
            nn.Linear(14,7))
        
        self.decoder1= nn.Sequential(
            nn.Linear(7,14),
            nn.Sigmoid(),
            nn.Linear(14,28),
            nn.LeakyReLU(),
            nn.Linear(28,42),
            nn.ReLU(),
            nn.Linear(42,56),
            nn.Sigmoid())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [23]:
model= autoencoder()
criterion= nn.MSELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
trainingLoss= []
validationLoss= []

In [24]:
#test the model
def testModel(loader):
    running_loss= 0.0
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        loss= criterion(output,inputs)
        running_loss += loss.item()
    
    return running_loss/nonFraudX_Valid.shape[0]
        

In [25]:
#train the model


for epoch in range(350):
    running_loss= 0.0
    for i, (feature,lable) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable)
        lables= lables.type(torch.LongTensor)
        
        # =====================forward====================
        output = model(inputs)
        loss= criterion(output,inputs)
     
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # =======print the statistics
        running_loss += loss.item()
        
    print('Epoch--[%d] --------------------Training loss: %.6f' %(epoch + 1,  running_loss /nonFraudX_trainNew.shape[0]))
    
    #test the model on the validation set.
    valid_loss= testModel(myNonFraudulentValidation)
    
    print('Epoch--[%d] --------------------Validation loss: %.6f' %(epoch + 1,  valid_loss))
    #append the loss after each epoch
    trainingLoss.append(running_loss/nonFraudX_trainNew.shape[0])
    validationLoss.append(valid_loss)
    
    

Epoch--[1] --------------------Training loss: 0.058048
Epoch--[1] --------------------Validation loss: 0.219872
Epoch--[2] --------------------Training loss: 0.049175
Epoch--[2] --------------------Validation loss: 0.162222
Epoch--[3] --------------------Training loss: 0.032672
Epoch--[3] --------------------Validation loss: 0.113978
Epoch--[4] --------------------Training loss: 0.027943
Epoch--[4] --------------------Validation loss: 0.109951
Epoch--[5] --------------------Training loss: 0.027500
Epoch--[5] --------------------Validation loss: 0.109199
Epoch--[6] --------------------Training loss: 0.027393
Epoch--[6] --------------------Validation loss: 0.108918
Epoch--[7] --------------------Training loss: 0.027337
Epoch--[7] --------------------Validation loss: 0.108790
Epoch--[8] --------------------Training loss: 0.027313
Epoch--[8] --------------------Validation loss: 0.108740
Epoch--[9] --------------------Training loss: 0.027299
Epoch--[9] --------------------Validation loss: 0

Epoch--[73] --------------------Validation loss: 0.108502
Epoch--[74] --------------------Training loss: 0.027253
Epoch--[74] --------------------Validation loss: 0.108480
Epoch--[75] --------------------Training loss: 0.027256
Epoch--[75] --------------------Validation loss: 0.108487
Epoch--[76] --------------------Training loss: 0.027251
Epoch--[76] --------------------Validation loss: 0.108495
Epoch--[77] --------------------Training loss: 0.027249
Epoch--[77] --------------------Validation loss: 0.108497
Epoch--[78] --------------------Training loss: 0.027252
Epoch--[78] --------------------Validation loss: 0.108484
Epoch--[79] --------------------Training loss: 0.027256
Epoch--[79] --------------------Validation loss: 0.108477
Epoch--[80] --------------------Training loss: 0.027251
Epoch--[80] --------------------Validation loss: 0.108482
Epoch--[81] --------------------Training loss: 0.027257
Epoch--[81] --------------------Validation loss: 0.108479
Epoch--[82] ------------------

Epoch--[145] --------------------Training loss: 0.027252
Epoch--[145] --------------------Validation loss: 0.108484
Epoch--[146] --------------------Training loss: 0.027254
Epoch--[146] --------------------Validation loss: 0.108479
Epoch--[147] --------------------Training loss: 0.027253
Epoch--[147] --------------------Validation loss: 0.108470
Epoch--[148] --------------------Training loss: 0.027257
Epoch--[148] --------------------Validation loss: 0.108486
Epoch--[149] --------------------Training loss: 0.027252
Epoch--[149] --------------------Validation loss: 0.108482
Epoch--[150] --------------------Training loss: 0.027253
Epoch--[150] --------------------Validation loss: 0.108487
Epoch--[151] --------------------Training loss: 0.027256
Epoch--[151] --------------------Validation loss: 0.108471
Epoch--[152] --------------------Training loss: 0.027256
Epoch--[152] --------------------Validation loss: 0.108491
Epoch--[153] --------------------Training loss: 0.027253
Epoch--[153] --

Epoch--[216] --------------------Training loss: 0.027254
Epoch--[216] --------------------Validation loss: 0.108478
Epoch--[217] --------------------Training loss: 0.027251
Epoch--[217] --------------------Validation loss: 0.108487
Epoch--[218] --------------------Training loss: 0.027253
Epoch--[218] --------------------Validation loss: 0.108479
Epoch--[219] --------------------Training loss: 0.027254
Epoch--[219] --------------------Validation loss: 0.108474
Epoch--[220] --------------------Training loss: 0.027249
Epoch--[220] --------------------Validation loss: 0.108477
Epoch--[221] --------------------Training loss: 0.027248
Epoch--[221] --------------------Validation loss: 0.108464
Epoch--[222] --------------------Training loss: 0.027257
Epoch--[222] --------------------Validation loss: 0.108480
Epoch--[223] --------------------Training loss: 0.027253
Epoch--[223] --------------------Validation loss: 0.108474
Epoch--[224] --------------------Training loss: 0.027251
Epoch--[224] --

Epoch--[287] --------------------Training loss: 0.027250
Epoch--[287] --------------------Validation loss: 0.108471
Epoch--[288] --------------------Training loss: 0.027252
Epoch--[288] --------------------Validation loss: 0.108467
Epoch--[289] --------------------Training loss: 0.027250
Epoch--[289] --------------------Validation loss: 0.108486
Epoch--[290] --------------------Training loss: 0.027252
Epoch--[290] --------------------Validation loss: 0.108482
Epoch--[291] --------------------Training loss: 0.027254
Epoch--[291] --------------------Validation loss: 0.108480
Epoch--[292] --------------------Training loss: 0.027248
Epoch--[292] --------------------Validation loss: 0.108488
Epoch--[293] --------------------Training loss: 0.027259
Epoch--[293] --------------------Validation loss: 0.108474
Epoch--[294] --------------------Training loss: 0.027251
Epoch--[294] --------------------Validation loss: 0.108476
Epoch--[295] --------------------Training loss: 0.027252
Epoch--[295] --

In [27]:
trainingLoss= pd.DataFrame(trainingLoss)
validationLoss= pd.DataFrame(validationLoss)

#save the dataframe
trainingLoss.to_csv('Batch4TrainingLoss.csv',index=None,sep='\t')
validationLoss.to_csv('Batch4ValidationLoss.csv',index=None,sep='\t')

In [ ]:
#function for testing the model
def testModel2(loader):
    predictedLabel= []
    lossList= []
    trueLable= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        
        trueLable.append(lable)
        
        loss= criterion(output,inputs)
        lossList.append(loss.item())
        
        if(loss.item()<0.01):
            predictedLabel.append(0)
        else:
            predictedLabel.append(1)
    return predictedLabel,lossList,trueLable


In [ ]:
#test the model on test set
predictedLable1,lossList1,trueLable1= testModel2(myNonFraudulentTestDataset)



In [ ]:
#count the predicted lables
class1= predictedLable1.count(1)
class0= predictedLable1.count(0)
print(class1)
print(class0)

In [ ]:
#test the model on test set
predictedLable2,lossList2,trueLable2= testModel2(myFraudulentTestDataset)


In [ ]:
#count the predicted lables
class1= predictedLable2.count(1)
class0= predictedLable2.count(0)
print(class1)
print(class0)

In [ ]:
#convert list to csv
nonFraudLoss= pd.DataFrame(lossList1)
fraudLoss= pd.DataFrame(lossList2)

In [ ]:
nonFraudLoss.head()

In [ ]:
#save the dataLoss:
nonFraudLoss.to_csv('nonFraudLoss.csv',index=None,sep='\t')
fraudLoss.to_csv('fraudLoss.csv',index=None,sep='\t')

In [ ]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)
